In [ ]:
from obsidianScripts.main import main
import json
from gptStuff.masterGPT import GPTFlow

json_data = None
with open('obsidianScripts/testJSONS/test1.json', 'r') as file:
    json_data = file.read()
    file.close()
json_data=json.loads(json_data)


main(json_data=json_data,debug=True)

In [ ]:
# get the token from the json
with open('token.json', 'r') as file:
    token = file.read()
    file.close()
token = json.loads(token)
token = token['token']
print(token)

In [ ]:
import requests

def set_webhook(bot_token, webhook_url):
    url = f'https://api.telegram.org/bot{bot_token}/setWebhook'
    payload = {
        'url': webhook_url
    }
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        print('Webhook set successfully!')
    else:
        print('Failed to set webhook.')
        print('Error:', response.text)

# Replace <YOUR_BOT_TOKEN> with your actual bot token
webhook_url = 'https://de50-140-82-220-240.ngrok-free.app'+'/telegram-webhook'

set_webhook(token, webhook_url)

In [ ]:
from flask import Flask, request
import subprocess

import requests
app = Flask(__name__)

@app.route('/telegram-webhook', methods=['POST'])
def webhook():
    # Get the input data from the request
    input_data = request.json

    # Extract the relevant information from the input data
    try:
        message_text = input_data['message']['text']
        chat_id = input_data['message']['chat']['id']
    except:
        print('Invalid input data:', input_data)
        return 'OK'

    # Run the Python script with the input data
    result = run_python_script(message_text)

    # Send the result back to Telegram
    send_message(chat_id, result)

    return 'OK'

def run_python_script(input_text):
    # Run the Python script as a subprocess and capture the output
    try:
        response = GPTFlow(input_text)
    except Exception as e:
        response = f'An error occurred: {e}'

    return response

def send_message(chat_id, text):
    # Send a message back to Telegram using the sendMessage method
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': text
    }
    requests.post(url, json=payload)

if __name__ == '__main__':
    print('Starting the Flask app')
    print('token:',token)
    app.run(port=3000)